# Since this file was supposed to run one time, this file is not optimzed and therefore takes long time to run

In [ ]:
import osmnx as ox
import networkx as nx
import numpy as np
import pandas as pd
import copy
import plotly.graph_objects as go

# Getting nodes and edges of the map

In [ ]:
# Enter the bounding box of required map
north, east, south, west = 33.93, -84.27, 33.6, -84.64
G = ox.graph_from_bbox(north, south, east, west, network_type = 'drive')

In [ ]:
# # read the existing file 
# G=ox.save_load.load_graphml(filename="/Users/apurvpriyam/Documents/CSE6242/Project/data/atlanta.graphml", folder=None)

In [ ]:
ox.plot_graph(G)

# Saving shortest path

In [ ]:
# ox.save_load.save_graphml(G, filename='Atlatna_med.graphml', folder=None, gephi=False)

In [ ]:
len(list(G.edges(data = True)))

# Updating the weights of node

In [ ]:
# Read the accident data
acc = pd.read_csv('/Users/apurvpriyam/Documents/CSE6242/Project/cse6242AP/inst/shiny/safe_path/data/US_Accidents_fil.csv')
acc.head

In [ ]:
# Creating a copy which will be filtered for requried area
accsmall = acc
accsmall.head

In [ ]:
# filtering for required area
accsmall = accsmall[accsmall['City'] == 'Atlanta']
accsmall = accsmall.reset_index()
print(accsmall.shape)

In [ ]:
# Adding accident attribure which will be updated
nx.set_edge_attributes(G, 0, 'accidents')
list(G.edges(data=True))[0]

In [ ]:
# Creating a dictionary storing number of accidents for each node
accidents_node = {}

for i in range(accsmall.shape[0]):
    if (i % 100 == 0):
        print(i)
    point = (accsmall['Start_Lat'][i], accsmall['Start_Lng'][i])
    node = ox.get_nearest_node(G, point)
    severity = accsmall['Severity'][i]
    if node in accidents_node:
        accidents_node[node]+=severity
    else:
        accidents_node[node]=severity
        
# accidents_node

accidents_list = []
for key in accidents_node:
    accidents_list.append(accidents_node[key])
    
cutoff = np.percentile(accidents_list, 95)

print(cutoff)
print(max(accidents_list))
for key in accidents_node:
    if accidents_node[key] > cutoff:
        accidents_node[key] = cutoff

In [ ]:
# Adding accident count for each edge

a = copy.deepcopy(list(G.edges(data=True)))
for i in range(len(G.edges)):
    if (i % 100 == 0):
        print(i)
        
    st_acc =0
    end_acc= 0
    if a[i][0] in accidents_node:
        st_acc = accidents_node[a[i][0]]
        
    if a[i][1] in accidents_node:
        end_acc = accidents_node[a[i][1]]
        
    list(G.edges(data=True))[i][2]['accidents'] = st_acc+end_acc
        
#     for key in accidents_node:
#         if (a[i][0] == key) | (a[i][1] == key):
#             list(G.edges(data=True))[i][2]['accidents']+=accidents_node[key]

In [ ]:
# acc_sum = 0
# for i in range(len(G.edges)):
#     acc_sum+=list(G.edges(data=True))[i][2]['accidents']
    
# print(acc_sum)
# sum(accsmall['Severity'])

# Creating different accidents weight 

In [ ]:
# normalizing the weights and accident

# fiding the max of length and accident

len_max = 0
acc_max = 0

a = copy.deepcopy(list(G.edges(data=True)))
for i in range(len(G.edges)):
    if (i % 100 == 0):
        print(i)
        
    if a[i][2]['length'] > len_max:
        len_max = list(G.edges(data=True))[i][2]['length']
    if a[i][2]['accidents'] > acc_max:
        acc_max = list(G.edges(data=True))[i][2]['accidents']

In [ ]:
print(len_max)
print(acc_max)

In [ ]:
# normalizing

nx.set_edge_attributes(G, 0, 'accidents_25')
nx.set_edge_attributes(G, 0, 'accidents_50')
nx.set_edge_attributes(G, 0, 'accidents_75')
nx.set_edge_attributes(G, 0, 'accidents_norm')

for i in range(len(G.edges)):
    if (i % 100 == 0):
        print(i)
        
    list(G.edges(data=True))[i][2]['length'] = list(G.edges(data=True))[i][2]['length']/len_max
    list(G.edges(data=True))[i][2]['accidents_norm'] = list(G.edges(data=True))[i][2]['accidents']/acc_max
    
    # adding extra attribute for different weights
    list(G.edges(data=True))[i][2]['accidents_25'] = 0.25*list(G.edges(data=True))[i][2]['accidents_norm']+0.75*list(G.edges(data=True))[i][2]['length']
    list(G.edges(data=True))[i][2]['accidents_50'] = 0.50*list(G.edges(data=True))[i][2]['accidents_norm']+0.50*list(G.edges(data=True))[i][2]['length']
    list(G.edges(data=True))[i][2]['accidents_75'] = 0.75*list(G.edges(data=True))[i][2]['accidents_norm']+0.25*list(G.edges(data=True))[i][2]['length']

In [ ]:
# saving the updated atlanta file
ox.save_load.save_graphml(G, filename='Atlanta_accidents_updt.graphml', folder=None, gephi=False)

# To check the weights by plotting different paths ~~~

In [ ]:
# Check
def node_list_to_path(G, node_list, use_geom=True):
    """
    Given a list of nodes, return a list of lines that together follow the path
    defined by the list of nodes.
    Parameters
    ----------
    G : networkx multidigraph
    route : list
        the route as a list of nodes
    use_geom : bool
        if True, use the spatial geometry attribute of the edges to draw
        geographically accurate edges, rather than just lines straight from node
        to node
    Returns
    -------
    lines : list of lines given as pairs ( (x_start, y_start), (x_stop, y_stop) )
    """
    edge_nodes = list(zip(node_list[:-1], node_list[1:]))
    lines = []
    for u, v in edge_nodes:
        # if there are parallel edges, select the shortest in length
        data = min(G.get_edge_data(u, v).values(), key=lambda x: x['length'])

        # if it has a geometry attribute (ie, a list of line segments)
        if 'geometry' in data and use_geom:
            # add them to the list of lines to plot
            xs, ys = data['geometry'].xy
            lines.append(list(zip(xs, ys)))
        else:
            # if it doesn't have a geometry attribute, the edge is a straight
            # line from node to node
            x1 = G.nodes[u]['x']
            y1 = G.nodes[u]['y']
            x2 = G.nodes[v]['x']
            y2 = G.nodes[v]['y']
            line = [(x1, y1), (x2, y2)]
            lines.append(line)
    return lines



In [ ]:
# define origin/desination points then get the nodes nearest to each
# from local to gatech
# origin_point = (33.786512, -84.401224)
# destination_point = (33.774755, -84.397217)

# # path on highway
# origin_point = (33.770052, -84.408432)
# destination_point = (33.770679, -84.385046)

# path across highway
origin_point = (33.786474, -84.401242)
destination_point = (33.779853, -84.384140)
# get the nearest nodes to the locations
origin_node = ox.get_nearest_node(G, origin_point)
destination_node = ox.get_nearest_node(G, destination_point)
origin_node, destination_node

In [ ]:
route = nx.shortest_path(G, origin_node, destination_node, weight = 'accidents_75')

In [ ]:
# getting the list of coordinates from the path (which is a list of nodes)
lines = node_list_to_path(G, route)

long = []
lat = []

for i in range(len(lines)):
    z = list(lines[i])
    l1 = list(list(zip(*z))[0])
    l2 = list(list(zip(*z))[1])
    for j in range(len(l1)):
        long.append(l1[j])
        lat.append(l2[j])

In [ ]:
# TODO: Set zoom dynamically

fig = go.Figure(go.Scattermapbox(
    name = "Path",
    mode = "lines",
    lon = long,
    lat = lat,
    marker = {'size': 10},
    line = dict(width = 4.5, color = 'blue'))) #'#1f77b4'),))


# Adding current location
current_loc = [-84.4006049, 33.7815131]
fig.add_trace(go.Scattermapbox(
    name = "Current Location",
    mode = "markers",
    lon = [current_loc[0]],
    lat = [current_loc[1]],
    marker = {'size': 12, 'color':'black'}))

# adding start and end
destination_point 
fig.add_trace(go.Scattermapbox(
    name = "Start",
    mode = "markers",
    lon = [origin_point[1]],
    lat = [origin_point[0]],
    marker = {'size': 12, 'color':"red"}))

fig.add_trace(go.Scattermapbox(
    name = "Destination",
    mode = "markers",
    lon = [destination_point[1]],
    lat = [destination_point[0]],
    marker = {'size': 12, 'color':'green'}))


# getting center for plots:
lat_center = np.mean(lat)
long_center = np.mean(long)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lat = 30, mapbox_center_lon=-80)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                 mapbox = {
                     'center': {'lat': lat_center, 'lon': long_center},  #(33.776770, -84.397)
                     #'center': {'lon': -20, 'lat': -20},
                     'zoom': 14})

fig.show()